# RECOPILACIÓN DE LO Q NOS DIO MEJOR RESULTADO (so far)

---
---

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, auc, roc_auc_score, roc_curve, RocCurveDisplay, PrecisionRecallDisplay, plot_precision_recall_curve, plot_roc_curve

In [8]:
df_orig = pd.read_csv(r"DS_G3_HeartFailure_FULL_train.csv",index_col=0)
df_cleaned = df_orig.drop(["time","anaemia","diabetes","high_blood_pressure","sex","smoking"],axis=1)

feat_train, feat_test, targ_train, targ_test = train_test_split(df_cleaned.drop(["DEATH_EVENT"],axis=1),df_cleaned["DEATH_EVENT"],test_size=0.2)

sss = StratifiedShuffleSplit(5) # USAMOS ESTO PARA CV POR EL DESBALANCE

sm = SMOTE()
feat_train, targ_train = sm.fit_resample(feat_train, targ_train)

## USAMOS UN RF PARA COMPROBAR LAS SUPOSICIONES DE IMPORTANCIA DE FEATURES

In [9]:
rf_model = RandomForestClassifier(n_estimators=5000)
rf_model.fit(feat_train,targ_train)
for feat in sorted(list(zip(rf_model.feature_importances_,df_cleaned.drop("DEATH_EVENT",axis=1).columns))):
    print(feat)

(0.12200226357422415, 'age')
(0.13111729342538173, 'serum_sodium')
(0.1391540078962218, 'creatinine_phosphokinase')
(0.14906799994550382, 'platelets')
(0.22606897249840807, 'serum_creatinine')
(0.23258946266026045, 'ejection_fraction')


---

In [10]:
hyperparam = {"n_estimators":[140,150,160],"max_depth":[100,110,120],"min_samples_leaf":[2,10,20,50]}
rf_model = RandomForestClassifier()
rf_grilla = GridSearchCV(rf_model,hyperparam,cv=sss,scoring="f1",n_jobs=-1)
rf_grilla.fit(feat_train,targ_train)
rf_model = rf_grilla.best_estimator_
print(rf_grilla.best_params_)
print("Grilla Best Score: {:.3f}".format(rf_grilla.best_score_))
rf_scoring = cross_val_score(rf_model, feat_train, targ_train,cv=sss, scoring="f1")
print("Cross Validation Score: {:.3f} +/- {:.3f}".format(rf_scoring.mean(),rf_scoring.std()))

{'max_depth': 100, 'min_samples_leaf': 2, 'n_estimators': 150}
Grilla Best Score: 0.824
Cross Validation Score: 0.793 +/- 0.059
